# 12 - Exporting Regression Output

Marina Adshade, Paul Corcuera, Giulia Lo Forte, Jane Platt  
2024-05-29

## Prerequisites

1.  Run OLS Regressions.

## Learning Outcomes

1.  Being able to export regression output in a table.
2.  Being able to plot regression coefficients in a graph.

## 12.0 Intro

In [1]:
import stata_setup
stata_setup.config('C:\Program Files\Stata18/','se')

In [2]:
>>> import sys
>>> sys.path.append('/Applications/Stata/utilities') # make sure this is the same as what you set up in Module 01, Section 1.3: Setting Up the STATA Path
>>> from pystata import config
>>> config.init('se')

## 12.1 Exporting Regression Output

When doing our project, presenting our results in a clear and organized
manner is as important as obtaining the results themselves. Stata’s
output is very clear on the computer display, but at some point we need
to “move” it from Stata to our draft. In this module, we will see how to
save a regression output in a table.

Once again, we will be using the fictional data set. Recall that this
data is simulating information of workers in the years 1982-2012 in a
fictional country where a training program was introduced in 2003 to
boost their earnings.

Let’s start by opening the dataset.

In [3]:
%%stata

* Load the dataset
clear *
*cd ""
use "fake_data.dta", clear

Imagine we are interested in estimating a multivariate regression of the
following form:

$$
\text{Earnings}_{it} = \alpha + \beta_1 \text{Age}_{it} + \beta_2 \text{Sex}_i + \varepsilon_{it}
$$

where $\text{Earnings}_{it}$ is the logarithm of earnings of individual
$i$ at time $t$, $\text{Age}_{it}$ is the logarithm of age of individual
$i$ at time $t$, and $\text{Sex}_i$ is a dummy variable equal to one if
the sex of individual $i$ is female.

First, we create the variables we need.

In [4]:
%%stata

* Create the variables
generate logearn = log(earnings)
generate logage = log(age)
generate sexdummy = 1 if sex == "F"
replace sexdummy = 0 if missing(sexdummy)

Then, we can estimate our specification using the command `regress`. We
have seen how to do it in [Module
11](https://comet.arts.ubc.ca/docs/Research/econ490-pystata/11_Linear_Reg.html).

In [5]:
%%stata

regress logearn logage sexdummy

There are different options available to export this table to another
file. In this module, we will use `etable`, a command available by
default in Stata 17 and subsequent versions.

`etable` can take several options. In its simplest form, we just need to
type `etable, export(filename)` after fitting a model to save a table in
a file named *filename*. We can use files of Microsoft Word, Microsoft
Excel, LATEX, Markdown, or PDF, but we need to specify the right
extension.

For example, let’s save our results in a Microsoft Word file named
*table.docx*.

In [6]:
%%stata

regress logearn logage sexdummy
etable, export(table.docx)

A file named *table.docx* should appear in your folder. Notice that this
worked, but our table does not have a very professional appearance yet.
We can add more options to the command `etable` to make our results more
clear and organized. Here are some of the options we can add:

-   we can add more statistics, such as the number of observations
    (*N*), the R$^2$ (*r2*), the adjusted R$^2$ (*r2_a*), and the F
    statistic (*F*), with the options `mstat(N)`, `mstat(r2)`,
    `mstat(r2_a)`, and `mstat(F)`;
-   we can add a title *titlename* with the option `title(titlename)`;
-   we can show the stars indicating the level of significance of our
    coefficients with the option `showstars` and add a footnote
    explaining them with `showstarsnote`;
-   for the coefficients, we can display the variable labels instead of
    their names by adding the option `varlabel`;
-   for the dependent variable, we can display its variable label
    instead of its name by adding the option `column(dvlabel)`;
-   we can show only some coefficients, by including them in
    `keep(coeffnames)`. For example, we can show only the coefficients
    for age and sex by adding the option `keep(logage sexdummy)`.

Let’s try all of them in practice. Notice that now we add the option
*replace* when we save the file because there is already a Microsoft
Word file named *table.docx*: `export(table.docx, replace)`.

In [7]:
%%stata

* Add labels to variables
label var logearn "Earnings (ln)"
label var logage "Age (ln)"
label var sexdummy "Female"

* Run regression
regress logearn logage sexdummy

* Store results
etable, export(table.docx, replace) mstat(N) mstat(r2_a) title(Earnings) showstars showstarsnote keep(logage sexdummy) varlabel column(dvlabel)

This is way nicer, but what if we want to show the results of multiple
models in the same table?

Suppose we want to first estimate a model with only *age* or only *sex*
as an explanatory variable, and then a multivariate model encompassing
both. In this case, we just need to store the results of each model
using the command `estimates store`.

In the example below, we store the three models in objects *model1*,
*model2*, and *model3*.

In [8]:
%%stata

* Store first regression in model1
regress logearn logage
estimates store model1

* Store second regression in model2
regress logearn sexdummy
estimates store model2

* Store third regression in model3
regress logearn logage sexdummy
estimates store model3

Now, we can export all the objects in one single table by calling their
names in the options `estimates()`.

In [9]:
%%stata

etable, estimates(model1 model2 model3) mstat(N) mstat(r2_a) showstars showstarsnote varlabel column(dvlabel) export(table.docx, replace)

## 12.2 Plotting Regression Coefficients

Visual representations can be better than tables. Sometimes we need to
plot our estimated coefficients and their confidence intervals.

In Stata, this is easily done with command `coefplot`. The graphs
obtained with `coefplot` are easy to customize. In its simplest use, we
only need to run `coefplot` right after our regression.

**Note:** You will need to install command `coefplot` from the SSC
Archive the first time you use it on your local computer. To do so, type
`ssc install coefplot`.

Once again, let’s try it on our multivariate model. We can omit the
constant by adding the option `drop(_cons)`. Remember to save the graph.

In [10]:
%%stata

regress logearn logage sexdummy
coefplot, drop(_cons)
graph export graph1.jpg, as(jpg) replace

Since it is a graph, we can add most of the options that we have seen in
[Module
9](https://comet.arts.ubc.ca/docs/Research/econ490-pystata/09_Stata_Graphs.html).
For example, we can change the color of the background from light blue
to white with the option `graphregion(color(white))`.

There are some options that are specific to `coefplot`. By default,
confidence intervals are drawn at 95% significance levels. We can
specify different and multiple levels in the option `levels()`. For
example, we can show both the 95% and 99.9% confidence intervals with
`levels(99.9 95)`.

Additionally, we can use a vertical layout with the option `vertical`.

Let’s apply these options to our example.

In [11]:
%%stata

regress logearn logage sexdummy
coefplot, drop(_cons) graphregion(color(white)) levels(99.9 95) vertical
graph export graph1.jpg, as(jpg) replace

## 12.3 Wrap Up

We have learned in this module how to store regression output in a clear
and organized manner using the command `etable` and how to plot
regression coefficients using the command `coefplot`.

Remember to check the Stata documentation when creating graphs and
exporting tables. The documentation can be your best ally if you end up
using it.

## 12.4 Wrap-up Table

| Command                    | Function                                                             |
|--------------------------------|----------------------------------------|
| `etable, export(filename)` | It exports the regression output to a file named *filename*.         |
| `coefplot`                 | It plots regression coefficients and their 95% confidence intervals. |

## References

[etable manual](https://www.stata.com/manuals/retable.pdf) <br> [How to
use
coefplot](https://repec.sowi.unibe.ch/stata/coefplot/getting-started.html)
<br>